In [9]:
import os
import pefile
import operator
import csv
import hashlib

from itertools import chain
from capstone import *
from capstone.x86 import *

class NGRAM_features:
    # N-gram 특징 추출 클래스의 생성자
    def __init__(self, output):

        self.output = output
        self.gram = dict()
        self.imports = ""

    # N-gram 생성 제너레이터
    def gen_list_n_gram(self, num, asm_list):

        for i in range(0, len(asm_list), num):
            yield asm_list[i:i+num]

    def n_grams(self, num, asm_list, ex_mode):
        if ex_mode == 1:
            gram = self.gram
        elif ex_mode == 0:
            gram = dict()

        gen_list = self.gen_list_n_gram(num, asm_list)

        for lis in gen_list:
            lis = " ".join(lis)
            try:
                gram[lis] += 1 # 명령어 패턴의 갯수를 하나 늘림
            except: # 해당 명령어 패턴이 gram에 없는 경우 예외 발생
                gram[lis] = 1 # 해당 명령어 패턴을 새로운 키 값으로 추가

        return gram


    def get_ngram_count(self, headers, grams, label):

        patterns = list()

        for pat in headers:
            try:
                patterns.append(grams[pat])
            except:
                patterns.append(0)

        patterns.append(label)

        return patterns


    # N-Gram을 구하기 위한 어셈블리 mnemonic 추출하기
    def get_opcodes(self, mode, file):
        asm = []
        byte_all = []
        try:
            # 파일의 PE헤더 구하기
            pe = pefile.PE(file)
        except:
            print("Error!!")
            return asm
        else:
            # OPTIONAL_HEADER의 AddressOfEntryPoint 값
            ep = pe.OPTIONAL_HEADER.AddressOfEntryPoint
            # OPTIONAL_HEADER의 SizeOfCode 값
            end = pe.OPTIONAL_HEADER.SizeOfCode
            # OPTIONAL_HEADER의 ImageBase 값
            ep_ava = ep+pe.OPTIONAL_HEADER.ImageBase

            for section in pe.sections:
                addr = section.VirtualAddress
                size = section.Misc_VirtualSize

                if ep > addr and ep < (addr+size):
                    #print(section.Name)
                    ep = addr
                    end = size

            data = pe.get_memory_mapped_image()[ep:ep+end] # 지정된 범위 안의 TEXT Dump 저장
            offset = 0

            temp = data.hex() # "/xab/xcd" 형식으로 저장된 데이터를
            temp = [temp[i:i+2] for i in range(0,len(temp), 2)] # "abcd" 형식으로 변경

            if(mode): # mode가 1(참)인 경우, temp를 반환함.
                return temp

            md = Cs(CS_ARCH_X86, CS_MODE_32) # Capstone 코드로, x86 디스어셈블링 모드로 전환
            md.detail = False

            for insn in md.disasm(data, 0x401000): # 분석하는 프로그램의 0x401000번지부터 디스어셈블링
                #print("0x%x:\t%s\t%s" % (insn.address, insn.mnemonic, insn.op_str))
                #print(insn.mnemonic)
                asm.append(insn.mnemonic)  # 추출한 어셈블리코드의 mnemonic만을 리스트에 추가

            return asm

    # SHA256 해시값 구하기
    def getSHA256(self, filepath):
        with open(filepath, 'rb') as fh:
            data = fh.read()
        return hashlib.sha256(data).hexdigest()

    # CSV파일 헤더 데이터 쓰기
    def write_csv_header(self, headers):
        filepath = self.output
        HASH = ['filename', 'SHA256']
        class_ = ['class']
        headers = HASH + headers + class_

        csv_file= open(filepath,"w")
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(headers)
        csv_file.close()

    # CSV파일에 데이터 쓰기
    def write_csv_data(self,data):
        filepath = self.output
        csv_file= open(filepath,"a")
        writer = csv.writer(csv_file, delimiter=',')
        writer.writerow(data)
        csv_file.close()


def main():

    # 몇 개 이상으로 나와야 특징으로 분류할지 지정
    num_of_features = 100

    # 분석할 파일이나 결과파일을 저장할 경로 지정
    nor_path = './'
    mal_path = './'
    output_file = "./ngram.csv"

    print('[*] Extracting ngram patterns from files')

    ef = NGRAM_features(output_file)
    i = 0

    # 악성 프로그램 4-gram 추출
    for file in os.listdir(mal_path):
        i += 1
        print("%d file processed (%s)," % (i, file))
        file = mal_path + file
        byte_code = ef.get_opcodes(0, file)
        grams = ef.n_grams(4, byte_code, 1)
        print("%d patterns extracted" % (len(grams)))

    print('- Malware Completed')

    # 정상 프로그램 4-gram 추출
    for file in os.listdir(nor_path):
        i += 1
        print("%d file processed (%s)," % (i, file))
        file = nor_path + file
        byte_code = ef.get_opcodes(0, file)
        grams = ef.n_grams(4, byte_code, 1)
        print("%d patterns extracted" % (len(grams)))
    print('- Normal Completed')

    # 상위 100개 패턴을 추출해 특징으로 만듦
    print("[*] Total length of 4-gram list :", len(grams))

    sorted_x = sorted(grams.items(), key=operator.itemgetter(1), reverse=True)
    print("[*] Using %s grams as features" % (num_of_features))
    features = sorted_x[0:num_of_features]
    headers = list(chain.from_iterable(zip(*features)))[0:num_of_features]
    ef.write_csv_header(headers)

    print("#" * 80)

    i = 0
    # 악성 프로그램의 4-Gram 정보를 CSV파일에 저장
    for file in os.listdir(mal_path):
        i += 1
        print("%d file processed (%s)," % (i, file))
        filepath = mal_path + file
        byte_code = ef.get_opcodes(0, filepath)
        grams = ef.n_grams(4, byte_code, 0)

        gram_count = ef.get_ngram_count(headers, grams, 1)
        hash_ = ef.getSHA256(filepath)
        all_data = [file, hash_]
        all_data.extend(gram_count)
        ef.write_csv_data(all_data)

    # 정상 프로그램의 4-Gram 정보를 CSV파일에 저장
    for file in os.listdir(nor_path):
        i += 1
        print("%d file processed (%s)," % (i, file))

        filepath = nor_path + file
        byte_code = ef.get_opcodes(0, filepath)
        grams = ef.n_grams(4, byte_code, 0)
        gram_count = ef.get_ngram_count(headers, grams, 0)

        hash_ = ef.getSHA256(filepath)
        all_data = [file, hash_]
        all_data.extend(gram_count)
        ef.write_csv_data(all_data)


if __name__ == '__main__':
    main()
    print("Get N-Gram Complete!!")

[*] Extracting ngram patterns from files
1 file processed (1. PE를 제외한 나머지 파일 삭제하기.ipynb),
Error!!
0 patterns extracted
2 file processed (2. SHA256 파일 이름 만들기.ipynb),
Error!!
0 patterns extracted
3 file processed (2. 수집한 악성코드 레이블링하기 (Using Virustotal API).ipynb),
Error!!
0 patterns extracted
4 file processed (3. PE_Value_Extraction.ipynb),
Error!!
0 patterns extracted
5 file processed (4. 악성코드 이미지 특징 분류.ipynb),
Error!!
0 patterns extracted
6 file processed (5. best-cnn-model.h5),
Error!!
0 patterns extracted
7 file processed (5. CNN_test.ipynb),
Error!!
0 patterns extracted
8 file processed (6. opcode N-Gram 분석.ipynb),
Error!!
0 patterns extracted
9 file processed (ngram.csv),
Error!!
0 patterns extracted
10 file processed (OfficeSetup.exe),
12691 patterns extracted
- Malware Completed
11 file processed (1. PE를 제외한 나머지 파일 삭제하기.ipynb),
Error!!
12691 patterns extracted
12 file processed (2. SHA256 파일 이름 만들기.ipynb),
Error!!
12691 patterns extracted
13 file processed (2. 수집한 악성코드 레이블링하기 (Usi